# Comparing taggers of Korean

@hyyoka

방대한 텍스트 데이터를 모두 수기로 형태소 태깅을 하는 것은 거의 불가능하다. 따라서, 자연어 처리에서는 자동으로 토큰들에 형태소를 태깅해주는 툴이 존재한다. 이렇게, 원시 말뭉치를 형태소 단위로 쪼개고 각각에 품사 정보를 부착하는 작업을 **형태소 분석(POS-tagging)**이라고 한다. 

이번 포스트에서는 nltk와 KoNLPy의 형태소 분석기를 이용해 영어 텍스트와 한국어 텍스트를 분석해보는 실습을 진행한다. 특히, KoNLPy에는 다양한 형태소 분석기가 존재하므로, 추가적인 2개의 형태소 분석기에 대한 간단한 비교도 진행한다. 

1. Okt(Open Korea Text) :: 전 Twitter
2. Hannanum
3. kkma(꼬고마)

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 45.8MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 460kB 57.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import nltk
import konlpy
from konlpy.tag import Hannanum, Okt, Kkma

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## 1. Dataload

한국어와 영어 각각에 대한 형태소 분석기의 성능을 비교하기 위해서는 bilingual parallel corpus가 필요하다고 판단했다. 검색해본 결과, 잘 구축된 Eng-Kor Parallel Corpus를 찾을 수 있었다. 사용한 코퍼스의 이름은 세종 영-한 코퍼스이다. 다음은 해당 코퍼스를 다운받은 [링크](http://corpus.mireene.com/download.html#)이다. 이 중, 랜덤한 4개의 샘플을 가져와 kor-eng.txt 라는 파일에 저장했다. 



In [ ]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = f.read()
        data = data.split("\n")
        data = [d for d in data if len(d)!=0]
    return data

data = read_data("/content/kor-eng.txt")

kor, eng = [], []
for i in range(len(data)):
    if i%2==0: kor.append(data[i])
    else: eng.append(data[i])

In [ ]:
kor = ['이승기의 앨범']

In [ ]:
eng

## 2. 형태소 분석

#### 2.1 ENG.TXT 형태소 분석


In [ ]:
eng_data = []
for s in eng: 
    tokens = nltk.word_tokenize(s.strip())
    eng_data.append(nltk.pos_tag(tokens))


### 2.2 KOR.TXT 형태소 분석

In [ ]:
okt = []
kkma = []
hannanum = []

for s in kor: 
    okt.append(Okt().pos(s))
    kkma.append(Kkma().pos(s))
    hannanum.append(Hannanum().pos(s))

## 3. 형태소 분석 결과

okt 형태소 분석기는 정말 단순한 결과만을 반환하므로 분석의 대상에 포함시키지 않았다. 

In [ ]:
df1 = pd.DataFrame([okt[0], kkma[0], hannanum[0]], index=["okt","kkma", "hannanum"])
df1

,0,1,2
okt,"(이승기, Noun)","(의, Josa)","(앨범, Noun)"
kkma,"(이승, NNG)","(기의, NNG)","(앨범, NNG)"
hannanum,"(이승기, N)","(의, J)","(앨범, N)"


첫 번째 문장을 확인해본 결과, 대체적으로 좋은 성능을 보였다. 

- 영어 텍스트의 경우 </br>
    Miracle을 NNP로 태깅한 것, Han River를 나눈 것, 또한 나누어 River를 NNP로 태깅한 것 외에는 큰 문제가 보이지 않았다. 해당 문제들은 모두 대문자로 시작한 것 때문에 NNP로 태깅한 것으로 보인다. 또한 Han River와 같은 여러 개의 단어로 이루어진 표현들도 문제가 되는 것으로 파악되었다. 

- 한국어 텍스트의 경우 </br>
     - kkma </br>
        라인강을 나눈 것, 이룩하다에서 이룩은 어근이지 명사+하다가 아니다. 
     - hannanum </br>
     마찬가지로 이룩을 명사로 본다. 




In [ ]:
df2 = pd.DataFrame([eng_data[1], okt[1], kkma[1], hannanum[1]], index=["Eng", "okt","kkma", "hannanum"])
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
Eng,"(During, IN)","(the, DT)","(past, JJ)","(two, CD)","(years, NNS)","(,, ,)","(Korea, NNP)","(was, VBD)","(hit, VBN)","(with, IN)","(a, DT)","(serious, JJ)","(economic, JJ)","(crisis, NN)","(that, WDT)","(swept, VBD)","(the, DT)","(Asian, JJ)","(region, NN)","(., .)","(Again, NNP)","(,, ,)","(Korea, NNP)","(has, VBZ)","(overcome, VBN)","(the, DT)","(difficulties, NNS)","(in, IN)","(cooperation, NN)","(with, IN)","(Germany, NNP)","(and, CC)","(many, JJ)","(other, JJ)","(friendly, JJ)","(countries, NNS)","(., .)",None,None,None,None,None,None
okt,"(한국, Noun)","(은, Josa)","(지난, Noun)","(2년, Number)","(동안, Noun)","(아시아, Noun)","(지역, Noun)","(을, Josa)","(휩쓴, Adjective)","(경제, Noun)","(적, Suffix)","(위기, Noun)","(를, Josa)","(국민, Noun)","(과, Josa)","(정부, Noun)","(의, Josa)","(헌신, Noun)","(,, Punctuation)","(그리고, Conjunction)","(독일, Noun)","(을, Josa)","(포함, Noun)","(한, Josa)","(국제사회, Noun)","(의, Josa)","(협력, Noun)","(에, Josa)","(힘, Noun)","(입어, Verb)","(성공, Noun)","(적, Suffix)","(으로, Josa)","(극복, Noun)","(하였습니다, Verb)","(., Punctuation)",None,None,None,None,None,None,None
kkma,"(한국, NNG)","(은, JX)","(지나, VV)","(ㄴ, ETD)","(2, NR)","(년, NNM)","(동안, NNG)","(아시아, NNG)","(지역, NNG)","(을, JKO)","(휩쓸, VV)","(ㄴ, ETD)","(경제적, NNG)","(위기, NNG)","(를, JKO)","(국민, NNG)","(과, JC)","(정부, NNG)","(의, JKG)","(헌신, NNG)","(,, SP)","(그리고, MAC)","(독일, NNG)","(을, JKO)","(포함, NNG)","(하, XSV)","(ㄴ, ETD)","(국제, NNG)","(사회, NNG)","(의, JKG)","(협력, NNG)","(에, JKM)","(힘입, VV)","(어, ECD)","(성공적, NNG)","(으로, JKM)","(극복, NNG)","(하, XSV)","(였, EPT)","(습니다, EFN)","(., SF)",None,None
hannanum,"(한국, N)","(은, J)","(지나, P)","(ㄴ, E)","(2년, N)","(동안, N)","(아시아, N)","(지역, N)","(을, J)","(휩쓸, P)","(ㄴ, E)","(경제적, N)","(위, N)","(이, J)","(기, E)","(를, J)","(국민, N)","(과, J)","(정부, N)","(의, J)","(헌신, N)","(,, S)","(그, N)","(이, J)","(리, E)","(고, J)","(독일, N)","(을, J)","(포함, N)","(하, X)","(ㄴ, E)","(국제사회, N)","(의, J)","(협력, N)","(에, J)","(힘입, P)","(어, E)","(성공적, N)","(으로, J)","(극복, N)","(하, X)","(었습니다, E)","(., S)"



두 번째 문장을 확인해본 결과, 영어는 여전히 좋은 성능을 보이지만, 한국어 형태소 분석기 중 hannanum에서 치명적인 오류가 발견되었다. 

- 영어 텍스트의 경우 </br>
Again은 adverb이다. 그런데 명사로 태깅되었다. 

- 한국어 텍스트의 경우
    - kkma </br>
    동안과 아시아가 동일하게 태깅되었다. -적의 경우, 명사와 -적을 나눌 필요가 있으나 형태소 분석 단위에서 이를 수행하지 않았다. 
    - hannanum </br>
    위기라는 명사를 위(N), 이(J), 기(E)로 태깅하였다. 
    그리고(M)을 그, 이, 리, 고로 나누어 태깅하였다. 


In [ ]:
df3 = pd.DataFrame([eng_data[2], okt[2], kkma[2], hannanum[2]], index=["Eng", "okt","kkma", "hannanum"])
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
Eng,"(When, WRB)","(the, DT)","(foreign, JJ)","(exchange, NN)","(crisis, NN)","(hit, VBD)","(Korea, NNP)","(at, IN)","(the, DT)","(end, NN)","(of, IN)","(1997, CD)","(,, ,)","(the, DT)","(foreign, JJ)","(exchange, NN)","(reserves, NNS)","(had, VBD)","(hit, VBN)","(rock, NN)","(bottom, NN)","(at, IN)","(US, NNP)","($, $)","(3.9, CD)","(billion, CD)","(;, :)","(in, IN)","(a, DT)","(matter, NN)","(of, IN)","(two, CD)","(years, NNS)","(,, ,)","(the, DT)","(figure, NN)","(now, RB)","(stands, VBZ)","(at, IN)","(a, DT)","(respectable, JJ)","(US, NNP)","($, $)","(80, CD)","(billion, CD)","(., .)"
okt,"(97년, Number)","(말, Noun)","(39억, Number)","(달러, Noun)","(에, Josa)","(불과했던, Adjective)","(외환보유고, Noun)","(는, Josa)","(이제, Noun)","(800억, Number)","(달러, Noun)","(에, Josa)","(도달, Noun)","(하였습니다, Verb)","(., Punctuation)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
kkma,"(97, NR)","(년, NNM)","(말, NNG)","(39, NR)","(억, NR)","(달러, NNG)","(에, JKM)","(불과, NNG)","(하, XSV)","(었, EPT)","(더, EPT)","(ㄴ, ETD)","(외환, NNG)","(보유고, NNG)","(는, JX)","(이제, MAG)","(800, NR)","(억, NR)","(달러, NNG)","(에, JKM)","(도달, NNG)","(하, XSV)","(였, EPT)","(습니다, EFN)","(., SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
hannanum,"(97년, N)","(말, P)","(ㄹ, E)","(39억, N)","(달러, N)","(에, J)","(불과, N)","(하, X)","(었던, E)","(외환보유고, N)","(는, J)","(이제, M)","(800억, N)","(달러, N)","(에, J)","(도달, N)","(하, X)","(었습니다, E)","(., S)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None



- 영어 텍스트의 경우
     큰 오류를 찾지 못하였다. 

- 한국어 텍스트의 경우
    - kkma
        과거를 나타내는 어미 '-던'을 더와 ㄴ으로 나누었다. 
    - hannanum
    end를 뜻하는 명사 '말'을 용언으로 태깅하였다. 또한 이를 용언으로 태깅해, ㄹ을 추가하였다. 

## 4. 평가

전반적으로 영어와 한국어 형태소 분석기 모두 성능이 괜찮다는 것을 확인할 수 있었다. 그러나, 가끔씩 오류가 발생하였다. 

특히 주목해야 할 점은 '-기'와 '-하다' 그리고 여러 개의 단어가 합쳐져 고유한 의미를 가지는 단어들이다. 위의 예시들을 보면 알 수 있듯, '위기'와 같은 오류는 위기를 위 + '-기'로 파악한 결과로 보이며, 예측하건데 명사형 전성어미로 파악한 것같다. 또한, 이룩하다를 어근 이룩과 하다가 아닌 도달하다와 같이 도달+하다로 해석하였다. 이는 모두 규칙 기반에 의한 것으로 보인다. 영어와 한국어의 공통적인 오류는 '한강'과 같은 명사를 나누는데서 발생했다. 

이에 대한 해결책으로는 사전 정보를 더욱 적극적으로 활용하는 것이다. '이룩'이라는 명사가 없으면 수정하는 식으로 개선할 수 있을 것 같다.